In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import bs4
import urllib3
import json
from datetime import tzinfo, timedelta, datetime
from pytz import timezone
from apscheduler.schedulers.blocking import BlockingScheduler

In [ ]:
with open("krx_code.json") as krx:
    krx_code = json.load(krx)

In [ ]:
with open("2018-02-14_less_than_15.json") as focus_group:
    focus = json.load(focus_group)

In [ ]:
def scrape_discussion(code):
    page_num = 1
    post_dic = {}
    post_num = 0
    click = 0
    like = 0
    dislike = 0
    unique_id = set()
    while True:
        target = "http://finance.naver.com/item/board.nhn?code=" + code + "&page=" + str(page_num)
        pm = urllib3.PoolManager()
        html = pm.urlopen(url=target, method="GET").data
        soup = bs4.BeautifulSoup(html, 'lxml')
        div_tag = soup.find_all("div", class_ ="section inner_sub")
        div_row = div_tag[0].find_all("tr")
        for row in div_row[2:25]:
            if div_row.index(row) not in [7, 13, 19]:
                info_list = row.find_all('td')
                time = info_list[0].text
                date, time = time.split(" ")
                date_list = date.split(".")
                time_list = time.split(":")
                pre_time = datetime(int(date_list[0]), int(date_list[1]), int(date_list[2]))
                korea_time = timezone('Asia/Seoul')
                pre_time = korea_time.localize(pre_time, is_dst = True)
                now = datetime.now(tz = timezone('Asia/Seoul'))
                if now - pre_time <= timedelta(days = 1):
                    post_num += 1
                    unique_id.add(info_list[3].text.strip("\n\t"))
                    click += int(info_list[4].text)
                    like += int(info_list[5].text)
                    dislike += int(info_list[6].text)
                else:
                    post_dic["post_num"] = post_num
                    post_dic["unique_id"] = len(unique_id)
                    post_dic["click"] = click
                    post_dic["like"] = like
                    post_dic["dislike"] = dislike

                    return post_dic
        
        page_num += 1
    
    return post_dic

In [ ]:
def job_function():
    import json
    
    rv = []
    tz = timezone('Asia/Seoul')
    seoul_now = datetime.now(tz)

    for stock in focus:
        d = scrape_discussion(krx_code[stock])
        d["name"] = stock
        d["time"] = str(seoul_now)[:16]
        rv.append(d)

    json = json.dumps(rv, ensure_ascii=False)
    f = open("discussion_" + str(seoul_now)[:16] + ".json","w")
    f.write(json)
    f.close()
    
    return None

In [ ]:
sched = BlockingScheduler()

In [ ]:
sched.add_job(job_function, 'cron', month='2', day='7-28', hour='0-23', \
              minute='0-59/10')

In [ ]:
sched.start()